In [1]:
import os
os.chdir("../")

In [2]:
from ngsolve import *
from netgen.geom2d import SplineGeometry

from netgen.occ import *
import netgen.meshing as ngm
import problems
import numpy as np
import params
import pickle
from time import time


In [3]:
## get problem parameters and geometry
problem = problems.problem1

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G = problem[0]['G']
geom = problem[1]
BC = problem[2]
h = 0.4
ord = 1
N = params.N
KBTV = params.KBTV
form = "HW" # EDP //Functional// HW

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    geo = OCCGeometry(geom)
    mesh = Mesh(geo.GenerateMesh(maxh=h))
    return mesh

mesh = mesher(geom, h)

def F(u):
    return Id(3) + Grad(u)
def Norm(vec):
    return InnerProduct(vec, vec)**0.5

def Gel_energy_functional(F):
    gamma = G/KBTV
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    return 0.5*gamma* Trace(F.trans*F ) + H

def Gel_energy_EDP(F): ## |F|^2 + H => gamma F:Gradv + H'*J'
    # ddet(A(t))/dt = det(A(t))*trace(A^-1(t)*Grad (v))
    gamma = G/KBTV
    J = Det(F)
    phi = phi0/J
    dv = Grad(v)
    invF = Inv(F)
    H_prime = -phi/N + log(1-phi) + phi + chi*phi**2
    edp = gamma * InnerProduct(F,dv) + H_prime * J * Trace(invF*dv)
    return edp

def HW_energy(F,v,P,q):
    # -P = \dfrac{\partial\phi_{FH}}{\partial F} 
    gamma = G/KBTV
    J = Det(F)
    phi = phi0/J
    H_prime = -phi/N + log(1-phi) + phi + chi*phi**2
    int1 = gamma*InnerProduct(F,Grad(v)) - P*J*Trace(Inv(F)*Grad(v))
    int2 = InnerProduct(-P-H_prime,q)
    return int1 + int2


# P = gamma*F + Hprime*  J*F^-T


In [4]:
## Generate spaces and forms
uSpace = VectorH1(mesh, order=ord+1, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])
PSpace = L2(mesh, order=ord)
# trabajo futuro probar los ordenes de los espacios para estabilidad
fes = FESpace([uSpace, PSpace])
u,P = fes.TrialFunction()
v,q = fes.TestFunction()
BF = BilinearForm(fes)
F = Id(3) + Grad(u)

## Assemble forms
def Assemble_Bilinear_Form(BF, F,v=None,P=None,q=None, form="Functional"):
    if form == "Functional":
        BF += Variation(Gel_energy_functional(F).Compile()*dx)
        return BF
    elif form == "EDP":
        BF += Gel_energy_EDP(F).Compile() * dx
        return BF
    elif form == "HW":
        BF += HW_energy(F,v,P,q)* dx
        return BF


BF = Assemble_Bilinear_Form(BF, F,v,P,q, form=form)

In [5]:
def Solver_freeswell(BF, gfu, tol=1e-8, maxiter=250, damp = 0.5):
    """
    Solves the problem
    """
    res = gfu.vec.CreateVector()
    w = gfu.vec.CreateVector()
    history = GridFunction(fes, multidim = 0)
    # here we may need to add another loop
   
    for iter in range(maxiter):
        # Prints before the iteration: number of it, residual, energy
        print("Energy: ", BF.Energy(gfu.vec), "Residual: ", sqrt(abs(InnerProduct(res,res))), "Iteration: ", iter)
        BF.Apply(gfu.vec, res)
        BF.AssembleLinearization(gfu.vec)
        inv = BF.mat.Inverse(freedofs = fes.FreeDofs())        
        w.data = damp * inv * res
        gfu.vec.data -= w
        history.AddMultiDimComponent(gfu.vec)
        if sqrt(abs(InnerProduct(w,res))) < tol:
            print("Converged")
            break
    return gfu, history


In [6]:
gfu = GridFunction(fes)
gfu.vec[:] = 0
gfu, history = Solver_freeswell(BF, gfu)

Energy:  0.0 Residual:  0.0 Iteration:  0
Energy:  0.03424339776665179 Residual:  0.05296457365140931 Iteration:  1
Energy:  0.031072394826292985 Residual:  0.03141262233868086 Iteration:  2
Energy:  0.02156622513975539 Residual:  0.018843189657851853 Iteration:  3
Energy:  0.013839957653867708 Residual:  0.011344294506399424 Iteration:  4
Energy:  0.008874064871069808 Residual:  0.0068104200888261645 Iteration:  5
Energy:  0.0058836463010303736 Residual:  0.004052685156820588 Iteration:  6
Energy:  0.003991810821579845 Residual:  0.0023757981074506826 Iteration:  7
Energy:  0.0026582385961011826 Residual:  0.0013633958470276994 Iteration:  8
Energy:  0.001671396422823438 Residual:  0.0007617190164218682 Iteration:  9
Energy:  0.00097898785423858 Residual:  0.0004132320874110554 Iteration:  10
Energy:  0.0005393965211133441 Residual:  0.00021811325638259813 Iteration:  11
Energy:  0.0002848643397740349 Residual:  0.00011267817449665703 Iteration:  12
Energy:  0.00014665465467507215 Res

In [7]:
from ngsolve.webgui import Draw
Draw(gfu.components[0], mesh, "u")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [8]:
vtk = VTKOutput(mesh, coefs=[gfu.components[0], gfu.components[1]], names=["u", "P"], filename=f"freeswell_HW_{h}")
vtk.Do()


'freeswell_HW_0.4'